# 📏 Depth Estimation с Depth Anything V2 + Segmentation

Этот notebook:
1. **Загружает Depth Anything V2 Metric** для оценки глубины в метрах
2. **Использует Mask R-CNN** для сегментации кубов и параллелепипедов
3. **Вычисляет расстояния** до каждого сегментированного объекта
4. **Определяет связи** между параллелепипедами и кубами
5. **Визуализирует** результаты с depth map и расстояниями

## 🎯 Параметры камеры из Unity

Для корректной метрической глубины используем параметры Unity камеры:
- **FOV**: 60° (вертикальный)
- **Resolution**: 1024×1024
- **Focal Length**: ~886.4 пикселей

## 1. Установка зависимостей

In [ ]:
# Depth Anything V2
!git clone https://github.com/DepthAnything/Depth-Anything-V2
!cd Depth-Anything-V2/metric_depth && pip install -r requirements.txt

# Segmentation
!pip install torch torchvision
!pip install pycocotools
!pip install opencv-python-headless
!pip install matplotlib
!pip install scipy
!pip install open3d

In [ ]:
import sys
sys.path.append('Depth-Anything-V2/metric_depth')

import os
import json
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from collections import defaultdict
import math

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from depth_anything_v2.dpt import DepthAnythingV2

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Конфигурация

In [ ]:
# Пути к данным
DATASET_PATH = "/kaggle/input/strawberry-peduncle-segmentation"
IMAGES_PATH = os.path.join(DATASET_PATH, "images")
ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "annotations.json")

# Путь к обученной Mask R-CNN
SEGMENTATION_MODEL_PATH = "/kaggle/input/cube-segmentation-model/best_model.pth"

# Unity Camera параметры
UNITY_FOV_VERTICAL = 60.0  # градусы
IMAGE_WIDTH = 1024
IMAGE_HEIGHT = 1024

# Вычисление focal length из Unity FOV
fov_rad = UNITY_FOV_VERTICAL * math.pi / 180.0
FOCAL_LENGTH_Y = (IMAGE_HEIGHT / 2.0) / math.tan(fov_rad / 2.0)
FOCAL_LENGTH_X = FOCAL_LENGTH_Y  # Square aspect ratio

print(f"📷 Unity Camera Intrinsics:")
print(f"   FOV: {UNITY_FOV_VERTICAL}°")
print(f"   Resolution: {IMAGE_WIDTH}×{IMAGE_HEIGHT}")
print(f"   Focal Length: {FOCAL_LENGTH_X:.2f} pixels")

# Depth Anything V2 настройки
DEPTH_ENCODER = 'vitl'  # vits, vitb, vitl
DEPTH_DATASET = 'hypersim'  # 'hypersim' для indoor, 'vkitti' для outdoor
MAX_DEPTH = 20.0  # 20m для indoor, 80m для outdoor

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Категории
CATEGORIES = {
    0: "background",
    1: "red_cube",
    2: "green_parallelepiped"
}

## 3. Загрузка моделей

In [ ]:
# Скачивание Depth Anything V2 Metric модели
depth_model_url = f"https://huggingface.co/depth-anything/Depth-Anything-V2-Metric-Hypersim-Large/resolve/main/depth_anything_v2_metric_hypersim_vitl.pth"
depth_model_path = "depth_anything_v2_metric_hypersim_vitl.pth"

if not os.path.exists(depth_model_path):
    print(f"Downloading Depth Anything V2 model...")
    !wget {depth_model_url} -O {depth_model_path}
else:
    print(f"Depth model already downloaded.")

In [ ]:
# Инициализация Depth Anything V2
model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]}
}

depth_model = DepthAnythingV2(**{**model_configs[DEPTH_ENCODER], 'max_depth': MAX_DEPTH})
depth_model.load_state_dict(torch.load(depth_model_path, map_location='cpu'))
depth_model = depth_model.to(DEVICE).eval()

print("✅ Depth Anything V2 loaded")

In [ ]:
# Загрузка Mask R-CNN для сегментации
def get_segmentation_model(num_classes):
    model = maskrcnn_resnet50_fpn(weights=None)
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    return model

segmentation_model = get_segmentation_model(num_classes=3)
segmentation_model.load_state_dict(torch.load(SEGMENTATION_MODEL_PATH, map_location='cpu'))
segmentation_model = segmentation_model.to(DEVICE).eval()

print("✅ Mask R-CNN loaded")

## 4. Функции для обработки

In [ ]:
def predict_depth(image_bgr):
    """
    Предсказывает depth map в метрах.
    
    Args:
        image_bgr: BGR изображение (OpenCV format)
    
    Returns:
        depth_map: (H, W) numpy array с глубиной в метрах
    """
    with torch.no_grad():
        depth = depth_model.infer_image(image_bgr, IMAGE_HEIGHT)
    
    return depth


def predict_segmentation(image_rgb, score_threshold=0.5):
    """
    Предсказывает сегментацию объектов.
    
    Args:
        image_rgb: RGB изображение (PIL/numpy format)
        score_threshold: Порог confidence
    
    Returns:
        prediction: dict с boxes, labels, scores, masks
    """
    # Конвертация в тензор
    if isinstance(image_rgb, np.ndarray):
        image_tensor = torch.as_tensor(image_rgb, dtype=torch.float32).permute(2, 0, 1) / 255.0
    else:
        image_tensor = torch.as_tensor(np.array(image_rgb), dtype=torch.float32).permute(2, 0, 1) / 255.0
    
    with torch.no_grad():
        prediction = segmentation_model([image_tensor.to(DEVICE)])[0]
    
    # Фильтрация по score
    keep = prediction['scores'] >= score_threshold
    filtered = {
        'boxes': prediction['boxes'][keep].cpu(),
        'labels': prediction['labels'][keep].cpu(),
        'scores': prediction['scores'][keep].cpu(),
        'masks': prediction['masks'][keep].cpu()
    }
    
    return filtered


def predict_associations(boxes, labels, scores):
    """
    Предсказывает связи между параллелепипедами и кубами.
    """
    boxes_np = boxes.numpy() if isinstance(boxes, torch.Tensor) else boxes
    labels_np = labels.numpy() if isinstance(labels, torch.Tensor) else labels
    scores_np = scores.numpy() if isinstance(scores, torch.Tensor) else scores
    
    cube_indices = np.where(labels_np == 1)[0]
    para_indices = np.where(labels_np == 2)[0]
    
    associations = {}
    
    for para_idx in para_indices:
        para_box = boxes_np[para_idx]
        para_center_x = (para_box[0] + para_box[2]) / 2
        para_bottom = para_box[3]
        
        min_dist = float('inf')
        best_cube_idx = None
        
        for cube_idx in cube_indices:
            cube_box = boxes_np[cube_idx]
            cube_center_x = (cube_box[0] + cube_box[2]) / 2
            cube_top = cube_box[1]
            
            vertical_dist = abs(para_bottom - cube_top)
            horizontal_dist = abs(para_center_x - cube_center_x)
            
            dist = np.sqrt(vertical_dist**2 + horizontal_dist**2)
            
            # Бонус если параллелепипед по центру куба
            if cube_box[0] <= para_center_x <= cube_box[2]:
                dist *= 0.5
            
            if dist < min_dist:
                min_dist = dist
                best_cube_idx = cube_idx
        
        associations[int(para_idx)] = int(best_cube_idx) if best_cube_idx is not None else None
    
    return associations

In [ ]:
def calculate_object_distance(depth_map, mask, boxes, idx):
    """
    Вычисляет расстояние до объекта используя depth map и маску.
    
    Args:
        depth_map: (H, W) depth в метрах
        mask: (H, W) бинарная маска объекта
        boxes: bounding boxes
        idx: индекс объекта
    
    Returns:
        distance_info: dict с min, max, mean, median расстояниями
    """
    # Извлекаем маску
    if isinstance(mask, torch.Tensor):
        obj_mask = mask[idx, 0].numpy() > 0.5
    else:
        obj_mask = mask > 0.5
    
    # Получаем глубины только для пикселей объекта
    object_depths = depth_map[obj_mask]
    
    if len(object_depths) == 0:
        return {
            'min': 0.0,
            'max': 0.0,
            'mean': 0.0,
            'median': 0.0,
            'center': 0.0
        }
    
    # Также вычисляем расстояние до центра bbox
    box = boxes[idx].numpy() if isinstance(boxes[idx], torch.Tensor) else boxes[idx]
    center_y = int((box[1] + box[3]) / 2)
    center_x = int((box[0] + box[2]) / 2)
    center_depth = depth_map[center_y, center_x]
    
    return {
        'min': float(np.min(object_depths)),
        'max': float(np.max(object_depths)),
        'mean': float(np.mean(object_depths)),
        'median': float(np.median(object_depths)),
        'center': float(center_depth)
    }

## 5. Полный inference pipeline

In [ ]:
def full_inference(image_path, visualize=True):
    """
    Полный pipeline: depth estimation + segmentation + associations + distances.
    
    Args:
        image_path: путь к изображению
        visualize: показывать ли визуализацию
    
    Returns:
        results: dict с depth_map, segmentation, associations, distances
    """
    # 1. Загрузка изображения
    image_rgb = np.array(Image.open(image_path).convert('RGB'))
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    
    print(f"📸 Processing: {os.path.basename(image_path)}")
    
    # 2. Depth estimation
    print("   🔍 Estimating depth...")
    depth_map = predict_depth(image_bgr)
    
    # 3. Segmentation
    print("   🎯 Segmenting objects...")
    segmentation = predict_segmentation(image_rgb)
    
    # 4. Associations
    print("   🔗 Predicting associations...")
    associations = predict_associations(
        segmentation['boxes'],
        segmentation['labels'],
        segmentation['scores']
    )
    
    # 5. Calculate distances for each object
    print("   📏 Calculating distances...")
    distances = []
    for i in range(len(segmentation['labels'])):
        dist_info = calculate_object_distance(
            depth_map,
            segmentation['masks'],
            segmentation['boxes'],
            i
        )
        distances.append(dist_info)
    
    # 6. Summary
    n_cubes = (segmentation['labels'] == 1).sum().item()
    n_paras = (segmentation['labels'] == 2).sum().item()
    
    print(f"\n   ✅ Results:")
    print(f"      Cubes: {n_cubes}")
    print(f"      Parallelepipeds: {n_paras}")
    print(f"      Associations: {len(associations)}")
    print(f"      Depth range: {depth_map.min():.2f}m - {depth_map.max():.2f}m")
    
    # 7. Visualization
    if visualize:
        visualize_results(
            image_rgb,
            depth_map,
            segmentation,
            associations,
            distances
        )
    
    return {
        'image': image_rgb,
        'depth_map': depth_map,
        'segmentation': segmentation,
        'associations': associations,
        'distances': distances
    }

## 6. Визуализация

In [ ]:
def visualize_results(image, depth_map, segmentation, associations, distances):
    """
    Визуализирует все результаты: RGB, depth, segmentation, distances.
    """
    fig, axes = plt.subplots(2, 2, figsize=(18, 16))
    
    boxes = segmentation['boxes'].numpy()
    labels = segmentation['labels'].numpy()
    scores = segmentation['scores'].numpy()
    masks = segmentation['masks'].numpy()
    
    # 1. Original RGB
    axes[0, 0].imshow(image)
    axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')
    
    # 2. Depth Map
    depth_vis = axes[0, 1].imshow(depth_map, cmap='turbo', vmin=0, vmax=MAX_DEPTH)
    axes[0, 1].set_title(f'Depth Map (0-{MAX_DEPTH}m)', fontsize=14, fontweight='bold')
    axes[0, 1].axis('off')
    plt.colorbar(depth_vis, ax=axes[0, 1], fraction=0.046, pad=0.04, label='Depth (m)')
    
    # 3. Segmentation with distances
    axes[1, 0].imshow(image)
    
    colors = {1: 'red', 2: 'green'}
    
    for i in range(len(labels)):
        box = boxes[i]
        label = labels[i]
        score = scores[i]
        dist = distances[i]
        color = colors.get(label, 'blue')
        
        # Draw bbox
        rect = plt.Rectangle(
            (box[0], box[1]), box[2] - box[0], box[3] - box[1],
            fill=False, edgecolor=color, linewidth=2
        )
        axes[1, 0].add_patch(rect)
        
        # Draw label with distance
        text = f"{CATEGORIES[label]}\n{dist['center']:.2f}m"
        axes[1, 0].text(
            box[0], box[1] - 10,
            text,
            color='white',
            fontsize=10,
            bbox=dict(boxstyle='round', facecolor=color, alpha=0.8)
        )
    
    axes[1, 0].set_title('Segmentation + Distances', fontsize=14, fontweight='bold')
    axes[1, 0].axis('off')
    
    # 4. Masks with associations
    axes[1, 1].imshow(image)
    
    # Draw masks
    combined_mask = np.zeros((*image.shape[:2], 3))
    for i in range(len(labels)):
        mask = masks[i, 0] > 0.5
        label = labels[i]
        
        if label == 1:  # Cube
            combined_mask[mask] = [1, 0, 0]
        elif label == 2:  # Parallelepiped
            combined_mask[mask] = [0, 1, 0]
    
    axes[1, 1].imshow(combined_mask, alpha=0.5)
    
    # Draw association lines
    for para_idx, cube_idx in associations.items():
        if cube_idx is not None:
            para_box = boxes[para_idx]
            cube_box = boxes[cube_idx]
            
            para_center = [(para_box[0] + para_box[2]) / 2, (para_box[1] + para_box[3]) / 2]
            cube_center = [(cube_box[0] + cube_box[2]) / 2, (cube_box[1] + cube_box[3]) / 2]
            
            axes[1, 1].plot(
                [para_center[0], cube_center[0]],
                [para_center[1], cube_center[1]],
                'y-', linewidth=2
            )
            axes[1, 1].plot(*para_center, 'yo', markersize=8)
            axes[1, 1].plot(*cube_center, 'yo', markersize=8)
    
    axes[1, 1].set_title('Masks + Associations', fontsize=14, fontweight='bold')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed distances
    print("\n📊 Detailed Distance Information:")
    print("=" * 80)
    for i in range(len(labels)):
        label_name = CATEGORIES[labels[i]]
        dist = distances[i]
        parent_info = ""
        
        if i in associations:
            parent_idx = associations[i]
            if parent_idx is not None:
                parent_dist = distances[parent_idx]
                parent_info = f" → Parent cube at {parent_dist['center']:.2f}m"
        
        print(f"[{i}] {label_name}:")
        print(f"    Center: {dist['center']:.3f}m | Mean: {dist['mean']:.3f}m | "
              f"Range: {dist['min']:.3f}m - {dist['max']:.3f}m{parent_info}")
    print("=" * 80)

## 7. Запуск inference на примерах

In [ ]:
# Получаем список изображений
import glob

image_files = sorted(glob.glob(os.path.join(IMAGES_PATH, "*.png")))[:5]

print(f"Found {len(image_files)} images for inference\n")

# Обрабатываем каждое изображение
all_results = []

for img_path in image_files:
    print("\n" + "="*80)
    results = full_inference(img_path, visualize=True)
    all_results.append(results)
    print("="*80)

## 8. Статистика по всем изображениям

In [ ]:
# Собираем статистику
all_cube_distances = []
all_para_distances = []

for result in all_results:
    labels = result['segmentation']['labels'].numpy()
    distances = result['distances']
    
    for i, label in enumerate(labels):
        if label == 1:  # Cube
            all_cube_distances.append(distances[i]['center'])
        elif label == 2:  # Parallelepiped
            all_para_distances.append(distances[i]['center'])

# Визуализация распределения расстояний
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(all_cube_distances, bins=20, color='red', alpha=0.7, edgecolor='black')
axes[0].set_title('Distance Distribution: Red Cubes', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Distance (m)')
axes[0].set_ylabel('Count')
axes[0].grid(True, alpha=0.3)

axes[1].hist(all_para_distances, bins=20, color='green', alpha=0.7, edgecolor='black')
axes[1].set_title('Distance Distribution: Green Parallelepipeds', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Distance (m)')
axes[1].set_ylabel('Count')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📈 Overall Statistics:")
print(f"   Cubes: {len(all_cube_distances)} detected")
print(f"      Mean distance: {np.mean(all_cube_distances):.3f}m")
print(f"      Std: {np.std(all_cube_distances):.3f}m")
print(f"      Range: {np.min(all_cube_distances):.3f}m - {np.max(all_cube_distances):.3f}m")
print(f"\n   Parallelepipeds: {len(all_para_distances)} detected")
print(f"      Mean distance: {np.mean(all_para_distances):.3f}m")
print(f"      Std: {np.std(all_para_distances):.3f}m")
print(f"      Range: {np.min(all_para_distances):.3f}m - {np.max(all_para_distances):.3f}m")

## 9. Экспорт результатов

In [ ]:
# Сохранение результатов в JSON
output_results = []

for i, (img_path, result) in enumerate(zip(image_files, all_results)):
    labels = result['segmentation']['labels'].numpy()
    boxes = result['segmentation']['boxes'].numpy()
    scores = result['segmentation']['scores'].numpy()
    distances = result['distances']
    associations = result['associations']
    
    objects = []
    for j in range(len(labels)):
        obj = {
            'id': j,
            'category': CATEGORIES[labels[j]],
            'bbox': boxes[j].tolist(),
            'score': float(scores[j]),
            'distance': {
                'center': distances[j]['center'],
                'mean': distances[j]['mean'],
                'min': distances[j]['min'],
                'max': distances[j]['max']
            }
        }
        
        if j in associations:
            obj['parent_id'] = associations[j]
        
        objects.append(obj)
    
    output_results.append({
        'image': os.path.basename(img_path),
        'depth_range': {
            'min': float(result['depth_map'].min()),
            'max': float(result['depth_map'].max()),
            'mean': float(result['depth_map'].mean())
        },
        'objects': objects
    })

# Сохранение
with open('depth_segmentation_results.json', 'w') as f:
    json.dump(output_results, f, indent=2)

print("✅ Results saved to depth_segmentation_results.json")

## 📝 Резюме

### Что делает этот notebook:

1. **Depth Estimation**: Использует Depth Anything V2 Metric для получения глубины в метрах
2. **Segmentation**: Mask R-CNN находит и сегментирует кубы и параллелепипеды
3. **Associations**: Геометрическая эвристика определяет связи
4. **Distance Calculation**: Для каждого объекта вычисляется расстояние до камеры

### Параметры Unity камеры:

- **FOV**: 60° → Focal Length: ~886.4 пикселей
- **Resolution**: 1024×1024
- Эти параметры используются для корректной метрической глубины

### Точность:

- Depth Anything V2 обучена на синтетических данных (Hypersim)
- Работает до 20м для indoor сцен
- Точность зависит от качества изображения и освещения

### Применение:

- Робототехника (навигация, захват объектов)
- AR/VR (размещение виртуальных объектов)
- Автономные системы (определение препятствий)
- Контроль качества (измерение размеров)